# shallow learning

In [1]:

import numpy as np
import pandas as pd 
import seaborn as sns  

import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from sklearn.metrics import mean_squared_error
# from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

import time	
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline 
from pylab import rcParams

rcParams['figure.dpi'] = 120
rcParams['savefig.dpi'] = 120 
rcParams['font.size'] = 20


from tqdm import tqdm

In [2]:
df = pd.read_csv('../datasets/sales_train_validation.csv')
print(df.shape)
df.head()

(30490, 1919)


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [3]:
submission = pd.read_csv('../datasets/sample_submission.csv')
print(submission.shape)
submission.head()

(60980, 29)


,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
price_df = pd.read_csv('../datasets/sell_prices.csv')
print(price_df.shape)
price_df.head()

(6841121, 4)


,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [5]:
cal_df = pd.read_csv('../datasets/calendar.csv')
print(cal_df.shape)
cal_df.head(5)

(1969, 14)


,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [6]:
cal_df["d"]=cal_df["d"].apply(lambda x: int(x.split("_")[1])) # d_1 to 1
cal_df.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,5,NaN,NaN,NaN,NaN,1,0,1


In [7]:
price_df["id"] = price_df["item_id"] + "_" + price_df["store_id"] + "_validation"

In [8]:
price_df.head()

,store_id,item_id,wm_yr_wk,sell_price,id
0,CA_1,HOBBIES_1_001,11325,9.58,HOBBIES_1_001_CA_1_validation
1,CA_1,HOBBIES_1_001,11326,9.58,HOBBIES_1_001_CA_1_validation
2,CA_1,HOBBIES_1_001,11327,8.26,HOBBIES_1_001_CA_1_validation
3,CA_1,HOBBIES_1_001,11328,8.26,HOBBIES_1_001_CA_1_validation
4,CA_1,HOBBIES_1_001,11329,8.26,HOBBIES_1_001_CA_1_validation


In [9]:
days_cols = df.columns[df.columns.str.contains("d_")].values
df_days = df[days_cols]
df_days

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,0,0,0,0,0,0,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,0,0,0,0,0,0,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,0,0,0,0,0,0,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,0,0,2,2,0,3,1,4,1,0,...,2,0,0,0,0,0,1,0,0,1
30486,0,0,0,0,0,5,0,1,1,3,...,0,0,0,0,0,0,0,0,1,0
30487,0,6,0,2,2,4,1,8,5,2,...,2,1,0,2,0,1,0,0,1,0
30488,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,3,1,3


## step 1
### *12 LEVEL 重要*

In [10]:
unit_sales_df =pd.DataFrame()

for day in tqdm(range(1858, 1886)):
    wk_id = list(cal_df[cal_df["d"]==day]["wm_yr_wk"])[0]
#     print("wk_id",wk_id)
    wk_price_df = price_df[price_df["wm_yr_wk"]==wk_id]
#     print("wk_price_df",wk_price_df)
    df = df.merge(wk_price_df[["sell_price", "id"]], on=["id"], how='inner')
#     df["unit_sales_" + str(day)] = df["sell_price"] * df["d_" + str(day)]
    unit_sales_df["unit_sales_" + str(day)] = df["sell_price"] * df["d_" + str(day)]
    df.drop(columns=["sell_price"], inplace=True)

100%|██████████| 28/28 [00:20<00:00,  1.36it/s]


In [11]:
df

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,2,0,0,0,0,0,1,0,0,1
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,2,1,0,2,0,1,0,0,1,0
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,1,0,0,1,0,3,1,3


In [13]:
df["dollar_sales"] = unit_sales_df.sum(axis=1)

In [14]:
print(df.shape)
print(df.columns)
df.head()

(30490, 1920)
Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd_1',
       'd_2', 'd_3', 'd_4',
       ...
       'd_1905', 'd_1906', 'd_1907', 'd_1908', 'd_1909', 'd_1910', 'd_1911',
       'd_1912', 'd_1913', 'dollar_sales'],
      dtype='object', length=1920)


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,dollar_sales
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,3,0,1,1,1,3,0,1,1,231.28
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,1,0,0,0,0,35.73
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,2,1,1,1,0,1,1,1,44.55
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,5,4,1,0,1,3,7,2,245.92
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,1,1,2,2,2,4,89.28


In [15]:
df["weight"] = df["dollar_sales"] / df["dollar_sales"].sum()

In [16]:
print(df.shape)
print(df.columns)
df.head()

(30490, 1921)
Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd_1',
       'd_2', 'd_3', 'd_4',
       ...
       'd_1906', 'd_1907', 'd_1908', 'd_1909', 'd_1910', 'd_1911', 'd_1912',
       'd_1913', 'dollar_sales', 'weight'],
      dtype='object', length=1921)


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,dollar_sales,weight
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,1,1,3,0,1,1,231.28,0.000063
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,1,0,0,0,0,35.73,0.000010
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,1,0,1,1,1,44.55,0.000012
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,5,4,1,0,1,3,7,2,245.92,0.000067
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,1,1,2,2,2,4,89.28,0.000024


In [17]:
df.drop(columns=["dollar_sales"], inplace=True)
df["weight"]/=12
print(df.shape)
print(df.columns)
df.head()

(30490, 1920)
Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd_1',
       'd_2', 'd_3', 'd_4',
       ...
       'd_1905', 'd_1906', 'd_1907', 'd_1908', 'd_1909', 'd_1910', 'd_1911',
       'd_1912', 'd_1913', 'weight'],
      dtype='object', length=1920)


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,weight
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,3,0,1,1,1,3,0,1,1,5.258191e-06
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,1,0,0,0,0,8.123278e-07
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,2,1,1,1,0,1,1,1,1.012852e-06
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,5,4,1,0,1,3,7,2,5.591034e-06
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,1,1,2,2,2,4,2.029797e-06


In [18]:
# df.to_csv("m_df.csv")

# ***

In [19]:
# df = pd.read_csv("m_df.csv",index_col=0)
df

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,weight
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,3,0,1,1,1,3,0,1,1,5.258191e-06
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,1,0,0,0,0,8.123278e-07
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,2,1,1,1,0,1,1,1,1.012852e-06
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,5,4,1,0,1,3,7,2,5.591034e-06
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,1,1,2,2,2,4,2.029797e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,0,0,0,0,0,1,0,0,1,4.697087e-07
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0.000000e+00
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,1,0,2,0,1,0,0,1,0,3.167010e-06
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,1,0,0,1,0,3,1,3,1.018536e-06


In [20]:
# regressor.fit(df_ml,df_ml_y)

In [21]:
# pred_df= pd.DataFrame(regressor.predict( 
#     df.drop(columns=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id',"weight"] + [c for c in df.columns if c.find("d_") == 0 and int(c.split("_")[1])not in range(train_start+28,train_end+28+1)])))

# pred_df.columns=["F_"+str(d) for d in  range(train_end+1,train_end+28+1)]

# # df+df.join(pred_df)

## step 2 
### *agg_df 12 level*

In [22]:
agg_df = pd.DataFrame(
#     df[[c for c in df.columns if c.find("d_") == 0 ]].sum()
    df[days_cols].sum()
    ).transpose()
agg_df

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,32631,31749,23783,25412,19146,29211,28010,37932,32736,25572,...,41789,48362,51640,38059,37570,35343,35033,40517,48962,49795


In [23]:
id_cols=["item_id",'dept_id','cat_id','store_id','state_id']
df[id_cols]
for col in id_cols:
    agg_df[col]='all'
agg_df

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1909,d_1910,d_1911,d_1912,d_1913,item_id,dept_id,cat_id,store_id,state_id
0,32631,31749,23783,25412,19146,29211,28010,37932,32736,25572,...,35343,35033,40517,48962,49795,all,all,all,all,all


In [24]:
agg_df['level'] =1
agg_df['weight'] =1/12
column_order =agg_df.columns
print(column_order)
agg_df

Index(['d_1', 'd_2', 'd_3', 'd_4', 'd_5', 'd_6', 'd_7', 'd_8', 'd_9', 'd_10',
       ...
       'd_1911', 'd_1912', 'd_1913', 'item_id', 'dept_id', 'cat_id',
       'store_id', 'state_id', 'level', 'weight'],
      dtype='object', length=1920)


,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1911,d_1912,d_1913,item_id,dept_id,cat_id,store_id,state_id,level,weight
0,32631,31749,23783,25412,19146,29211,28010,37932,32736,25572,...,40517,48962,49795,all,all,all,all,all,1,0.083333


In [25]:
level_groupings = {2: ["state_id"], 3: ["store_id"], 4: ["cat_id"], 5: ["dept_id"], 
              6: ["state_id", "cat_id"], 7: ["state_id", "dept_id"], 8: ["store_id", "cat_id"], 9: ["store_id", "dept_id"],
              10: ["item_id"], 11: ["item_id", "state_id"]}

In [26]:
agg_df

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1911,d_1912,d_1913,item_id,dept_id,cat_id,store_id,state_id,level,weight
0,32631,31749,23783,25412,19146,29211,28010,37932,32736,25572,...,40517,48962,49795,all,all,all,all,all,1,0.083333


In [27]:
df.groupby(by=level_groupings[11]).sum() # 12级别分类

d_1  d_2  d_3  d_4  d_5  d_6  d_7  d_8  d_9  d_10  \
item_id         state_id                                                      
FOODS_1_001     CA          6    3    2    3    7    5    8    3    5     2   
                TX          0    1    2    2    0    4    0    0    4     3   
                WI          0    2    0    1    0    9    2    1    2     5   
FOODS_1_002     CA          3    3    4    4    3    3    0    2    1     1   
                TX          0    0    2    0    0    0    0    1    0     0   
...                       ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   
HOUSEHOLD_2_515 TX          0    0    0    0    0    0    0    0    0     0   
                WI          0    0    0    0    0    0    0    0    0     0   
HOUSEHOLD_2_516 CA          0    2    0    1    1    1    0    2    1     2   
                TX          2    1    0    0    0    0    1    1    0     0   
                WI          0    1    0    2    0    1    1    1    0     0   

                          ...  d_1905  d_1906  d_1907  d_1908  d_1909  d_1910  \
item_id         state_id  ...                                                   
FOODS_1_001     CA        ...       2      27       6       2       2       4   
                TX        ...       1       2       1       0       0       2   
                WI        ...       1       1       0       3       1       0   
FOODS_1_002     CA        ...       5       2       0       0       2       2   
                TX        ...       1       0       0       2       1       0   
...                       ...     ...     ...     ...     ...     ...     ...   
HOUSEHOLD_2_515 TX        ...       0       0       0       0       0       0   
                WI        ...       1       0       0       0       1       1   
HOUSEHOLD_2_516 CA        ...       0       0       0       0       0       1   
                TX        ...       1       1       0       0       1       0   
                WI        ...       0       0       0       0       0       1   

                          d_1911  d_1912  d_1913        weight  
item_id         state_id                                        
FOODS_1_001     CA             2       3       5  4.990826e-06  
                TX             0       1       0  2.342633e-06  
                WI             0      12       1  1.273170e-06  
FOODS_1_002     CA             1       2       1  1.293177e-05  
                TX             0       0       0  3.017413e-06  
...                          ...     ...     ...           ...  
HOUSEHOLD_2_515 TX             0       0       0  2.239415e-07  
                WI             1       1       0  4.478830e-07  
HOUSEHOLD_2_516 CA             1       0       1  2.295798e-06  
                TX             0       0       0  2.430845e-06  
                WI             0       0       0  6.752347e-07  

[9147 rows x 1914 columns]

In [28]:
for level in tqdm(level_groupings):
    temp_df = df.groupby(by=level_groupings[level]).sum().reset_index()
    temp_df["level"] = level
#     temp_df["weight"] /= 12
    for c in column_order:
        if c not in temp_df.columns:
            temp_df[c] = "all"
    agg_df = agg_df.append(temp_df[column_order])

del temp_df

100%|██████████| 10/10 [00:17<00:00,  1.74s/it]


In [29]:
print(agg_df.shape)
print(agg_df.columns)
agg_df
# agg_df.head(40)

(12350, 1920)
Index(['d_1', 'd_2', 'd_3', 'd_4', 'd_5', 'd_6', 'd_7', 'd_8', 'd_9', 'd_10',
       ...
       'd_1911', 'd_1912', 'd_1913', 'item_id', 'dept_id', 'cat_id',
       'store_id', 'state_id', 'level', 'weight'],
      dtype='object', length=1920)


,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1911,d_1912,d_1913,item_id,dept_id,cat_id,store_id,state_id,level,weight
0,32631,31749,23783,25412,19146,29211,28010,37932,32736,25572,...,40517,48962,49795,all,all,all,all,all,1,8.333333e-02
0,14195,13805,10108,11047,9925,11322,12251,16610,14696,11822,...,17095,21834,23187,all,all,all,all,CA,2,3.614587e-02
1,9438,9630,6778,7381,5912,9006,6226,9440,9376,7319,...,10615,12266,12282,all,all,all,all,TX,2,2.328074e-02
2,8998,8314,6897,6984,3309,8883,9533,11882,8664,6431,...,12807,14862,14326,all,all,all,all,WI,2,2.390672e-02
0,4337,4155,2816,3051,2630,3276,3450,5437,4340,3157,...,4387,5577,6113,all,all,all,CA_1,all,3,9.091642e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9142,0,0,0,0,0,0,0,0,0,0,...,0,0,0,HOUSEHOLD_2_515,all,all,all,TX,11,2.239415e-07
9143,0,0,0,0,0,0,0,0,0,0,...,1,1,0,HOUSEHOLD_2_515,all,all,all,WI,11,4.478830e-07
9144,0,2,0,1,1,1,0,2,1,2,...,1,0,1,HOUSEHOLD_2_516,all,all,all,CA,11,2.295798e-06
9145,2,1,0,0,0,0,1,1,0,0,...,0,0,0,HOUSEHOLD_2_516,all,all,all,TX,11,2.430845e-06


In [30]:
print(df.shape[0], agg_df.shape[0], df.shape[0] + agg_df.shape[0])


30490 12350 42840


In [31]:
agg_df["weight"].sum()  + df["weight"].sum()


1.0000000000000007

In [32]:
# agg_df.to_csv("agg_df_12level.csv")

In [33]:
# agg_df=pd.read_csv("agg_df_12level.csv",index_col=0)

## ML

In [34]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
import random


In [35]:
h = 28
def rmsse(ground_truth, forecast, train_series, axis=1, n=1885):
    # assuming input are numpy array or matrices
    assert axis == 0 or axis == 1
    assert type(ground_truth) == np.ndarray and type(forecast) == np.ndarray and type(train_series) == np.ndarray
    
    if axis == 1:
        # using axis == 1 we must guarantee these are matrices and not arrays
        assert ground_truth.shape[1] > 1 and forecast.shape[1] > 1 and train_series.shape[1] > 1
    
    numerator = ((ground_truth - forecast)**2).sum(axis=axis)
    if axis == 1:
        denominator = 1/(n-1) * ((train_series[:, 1:] - train_series[:, :-1]) ** 2).sum(axis=axis)
    else:
        denominator = 1/(n-1) * ((train_series[1:] - train_series[:-1]) ** 2).sum(axis=axis)
    return (1/h * numerator/denominator) ** 0.5

In [36]:
pd.get_dummies(df.drop(columns=["id", "item_id", "weight"]))

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,store_id_CA_4,store_id_TX_1,store_id_TX_2,store_id_TX_3,store_id_WI_1,store_id_WI_2,store_id_WI_3,state_id_CA,state_id_TX,state_id_WI
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,0,0,2,2,0,3,1,4,1,0,...,0,0,0,0,0,0,1,0,0,1
30486,0,0,0,0,0,5,0,1,1,3,...,0,0,0,0,0,0,1,0,0,1
30487,0,6,0,2,2,4,1,8,5,2,...,0,0,0,0,0,0,1,0,0,1
30488,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1


In [37]:
df = df[["id", "item_id", "dept_id", "cat_id", "store_id", "state_id", "weight"]].join(pd.get_dummies(df.drop(columns=["id", "item_id", "weight"])))

In [38]:
df

,id,item_id,dept_id,cat_id,store_id,state_id,weight,d_1,d_2,d_3,...,store_id_CA_4,store_id_TX_1,store_id_TX_2,store_id_TX_3,store_id_WI_1,store_id_WI_2,store_id_WI_3,state_id_CA,state_id_TX,state_id_WI
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,5.258191e-06,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,8.123278e-07,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1.012852e-06,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,5.591034e-06,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,2.029797e-06,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,4.697087e-07,0,0,2,...,0,0,0,0,0,0,1,0,0,1
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0.000000e+00,0,0,0,...,0,0,0,0,0,0,1,0,0,1
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,3.167010e-06,0,6,0,...,0,0,0,0,0,0,1,0,0,1
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,1.018536e-06,0,0,0,...,0,0,0,0,0,0,1,0,0,1


In [39]:
best_s = 100
best_m = None
best_start_date = 1000

In [ ]:
for _ in tqdm(range(50)):
    rand_est = random.randint(20, 50)
    rand_depth = random.randint(10, 30)
    rand_start_date = random.randint(1200, 1500)
    
    print(rand_est, rand_depth)
    
    
    average = []
    
    for cv in range(1, 4):
        train_start = rand_start_date - 28 * cv
        train_end = 1885 - 28 * cv
        
        regressor = ExtraTreesRegressor(n_estimators=rand_est, max_depth=rand_depth, random_state=42)
        
        drop_cols = [item for item in [c for c in df.columns if c.find("F_")==0] + ['wrmsse', 'rmsse'] if item in df.columns]
        df.drop(columns=drop_cols, inplace=True)

        regressor.fit(df.drop(columns=["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"] +\
                              [c for c in df.columns if c.find("d_")==0 and int(c.split("_")[1]) not in range(train_start, train_end + 1)]),
              df[[c for c in df.columns if c.find("d_")==0 and int(c.split("_")[1]) in range(train_end + 1, train_end + 28 + 1)]])

        pred_df = pd.DataFrame(regressor.predict(df.drop(columns=["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"] +\
                                       [c for c in df.columns if c.find("d_")==0 and int(c.split("_")[1]) not in range(train_start+28, train_end + 28 + 1)])))
        pred_df.columns = ["F_" + str(d) for d in range(train_end + 28 + 1, train_end + 28 + 28 + 1)]
        df = df.join(pred_df)

        # remake agg_df
        new_agg_df = pd.DataFrame(df[[c for c in df.columns if c.find("d_") == 0 or c.find("F_") == 0]].sum()).transpose()
        new_agg_df["level"] = 1
        new_agg_df["weight"] = 1/12
        column_order = new_agg_df.columns

        for level in level_groupings:
            temp_df = df.groupby(by=level_groupings[level]).sum().reset_index()
            temp_df["level"] = level
            new_agg_df = new_agg_df.append(temp_df[column_order])
        del temp_df

        agg_df = new_agg_df
        
        train_series_cols = [c for c in df.columns if c.find("d_") == 0][:-28]
        ground_truth_cols = [c for c in df.columns if c.find("d_") == 0][-28:]
        forecast_cols = [c for c in df.columns if c.find("F_") == 0]

        df["rmsse"] = rmsse(np.array(df[ground_truth_cols]), 
                np.array(df[forecast_cols]), np.array(df[train_series_cols]))
        agg_df["rmsse"] = rmsse(np.array(agg_df[ground_truth_cols]), 
                np.array(agg_df[forecast_cols]), np.array(agg_df[train_series_cols]))

        df["wrmsse"] = df["weight"] * df["rmsse"]
        agg_df["wrmsse"] = agg_df["weight"] * agg_df["rmsse"]

        print("CV", cv, ":", df["wrmsse"].sum() + agg_df["wrmsse"].sum())

        average.append(df["wrmsse"].sum() + agg_df["wrmsse"].sum())
    
    this_s = np.array(average).mean()
    if this_s < best_s:
        best_s = this_s
        best_m = regressor
        best_start_date = rand_start_date
        
    print(this_s, best_s)

  0%|          | 0/50 [00:00<?, ?it/s]

36 15
CV 1 : 0.7836962141263903
CV 2 : 0.866442054218693


  2%|▏         | 1/50 [05:46<4:42:46, 346.25s/it]

CV 3 : 0.8891817109041317
0.8464399930830716 0.8464399930830716
22 25
CV 1 : 0.7714919907299541
CV 2 : 0.8402991482171679


  4%|▍         | 2/50 [12:22<4:49:04, 361.34s/it]

...

CV 3 : 0.8943081005780146
0.8482187117444925 0.8167864914902646
44 23
CV 1 : 0.76987859288037
CV 2 : 0.8383131773401781


 46%|████▌     | 23/50 [2:51:08<3:46:42, 503.80s/it]

CV 3 : 0.8659436687556484
0.8247118129920654 0.8167864914902646
25 23
CV 1 : 0.7701785858502376
CV 2 : 0.8443405032834363


 48%|████▊     | 24/50 [2:58:26<3:29:39, 483.83s/it]

CV 3 : 0.865681111134456
0.8267334000893767 0.8167864914902646
38 23
CV 1 : 0.7656104351480494
CV 2 : 0.8397610208506534


In [ ]:
# fit the best_m with the closest training set
best_m.fit(df.drop(columns=["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"] +\
                              [c for c in df.columns if c.find("d_")==0 and int(c.split("_")[1]) not in range(best_start_date, 1886)]),
              df[[c for c in df.columns if c.find("d_")==0 and int(c.split("_")[1]) in range(1886, 1914)]])

In [ ]:
#x_train
df.drop(columns=["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"] +\
                              [c for c in df.columns if c.find("d_")==0 and int(c.split("_")[1]) not in range(best_start_date, 1886)])

In [ ]:
#y-train
df[[c for c in df.columns if c.find("d_")==0 and int(c.split("_")[1]) in range(1886, 1914)]]

In [ ]:
#x_pred
df.drop(columns=["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"] +
                               [c for c in df.columns if c.find("d_")==0 and int(c.split("_")[1]) not in range(best_start_date+28, 1914)])

In [ ]:
submit_df = df[["id"]]
pred = best_m.predict(df.drop(columns=["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"] +
                               [c for c in df.columns if c.find("d_")==0 and int(c.split("_")[1]) not in range(best_start_date+28, 1914)]))
for i in range(1, 29):
    submit_df["F" + str(i)] = pred[:, i-1]

In [ ]:
submit_df2 = submit_df.copy()
submit_df2["id"] = submit_df2["id"].apply(lambda x: x.replace('validation',
                                                              'evaluation'))


In [ ]:
submit_df = submit_df.append(submit_df2).reset_index(drop=True)


In [ ]:
submit_df.to_csv("submission-v5.csv", index=False)


In [ ]:
submit_df